<a href="https://colab.research.google.com/github/andressa-uchoas/pretalab/blob/main/andressa_uchoas_atividade_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Projeto 2: Banco de Dados

Objetivo: Entender a jornada do cliente desde a navegação no site até a reserva e sua experiência final.

Executar:

1. Inserir os dados dos arquivos csv em um banco de dados SQLite.

2. Realizar de consultas para responder às perguntas de análise.

3. Salvar as consultas como tabela no banco de dados

Responder:

1. Qual o tempo médio que os clientes gastam no site antes de fazer uma reserva?

2. Qual o percentual de clientes que realizam reservas após a navegação?

3. Existe alguma correlação entre o tempo gasto no site e a avaliação dada?

In [173]:
# Importar bibliotecas
import pandas as pd
import sqlite3

In [198]:
# Conectar ao banco de dados
conn = sqlite3.connect('my_db.db')

In [199]:
# Criar um cursor para executar comandos SQL
cursor = conn.cursor()

In [176]:
# Criar tabelas
#id_cliente	nome	email	idade	genero
cursor.execute('''
CREATE TABLE cliente (
    id_cliente INTEGER PRIMARY KEY AUTOINCREMENT,
    nome TEXT,
    email TEXT,
    idade INTEGER,
    genero TEXT
)
''')

#id_navegacao	id_cliente	pagina	tempo_gasto	data_acesso
cursor.execute('''
CREATE TABLE navegacao (
    id_navegacao INTEGER PRIMARY KEY AUTOINCREMENT,
    id_cliente INTEGER,
    pagina TEXT,
    tempo_gasto INTEGER,
    data_acesso DATE
)
''')

#id_avaliacao	id_cliente	nota	comentario
cursor.execute('''
CREATE TABLE avaliacao (
    id_avaliacao INTEGER PRIMARY KEY AUTOINCREMENT,
    id_cliente INTEGER,
    nota REAL,
    comentario TEXT
)
''')

#id_reserva	id_cliente	data_checkin	data_checkout	valor_total
cursor.execute('''
CREATE TABLE reserva (
    id_reserva INTEGER PRIMARY KEY AUTOINCREMENT,
    id_cliente INTEGER,
    data_checkin DATE,
    data_checkout DATE,
    valor_total REAL
)
''')

In [177]:
# Exibir as tabelas existentes
cursor.execute("select name from sqlite_master where type='table';")
tabelas = cursor.fetchall()
print(tabelas)

[('sqlite_sequence',), ('cliente',), ('navegacao',), ('avaliacao',), ('reserva',)]


In [190]:
# Se necessário, deletar as tabelas existentes
#cursor.execute('DROP TABLE IF EXISTS cliente')
#cursor.execute('DROP TABLE IF EXISTS navegacao')
#cursor.execute('DROP TABLE IF EXISTS avaliacao')
#cursor.execute('DROP TABLE IF EXISTS reserva')

In [179]:
# Importar os dados para as tabelas criadas

base_cliente = pd.read_csv('/content/clientes.csv', delimiter=',',header=0, )
base_navegacao = pd.read_csv('/content/navegacao.csv', delimiter=',',header=0, )
base_avaliacao = pd.read_csv('/content/avaliacoes.csv', delimiter=',',header=0, )
base_reserva = pd.read_csv('/content/reservas.csv', delimiter=',',header=0, )

In [180]:
# Inserir os dados nas tabelas

base_cliente.to_sql('cliente', conn, if_exists='replace', index=False)
base_navegacao.to_sql('navegacao', conn, if_exists='replace', index=False)
base_avaliacao.to_sql('avaliacao', conn, if_exists='replace', index=False)
base_reserva.to_sql('reserva', conn, if_exists='replace', index=False)

base_navegacao['data_acesso'] = pd.to_datetime(base_navegacao['data_acesso'])
base_reserva[['data_checkin', 'data_checkout']] = base_reserva[['data_checkin', 'data_checkout']].apply(pd.to_datetime)

Qual o tempo médio que os clientes gastam no site antes de fazer uma reserva?

In [193]:
cursor.execute('''

select

round(avg(n.tempo_gasto),0) as avg_before_checkin

from navegacao n
left join reserva r on r.id_cliente = n.id_cliente
where date(r.data_checkin) > date(n.data_acesso)
''')

resultados = cursor.fetchall()
for resultado in resultados:
    dias = resultado[0]
    print(f"O tempo médio antes da reserva é de {int(dias)} dias.")

O tempo médio antes da reserva é de 60 dias.


Qual o percentual de clientes que realizam reservas após a navegação?

In [182]:
cursor.execute ('''

select

count(distinct r.id_cliente)*100/count(distinct n.id_cliente) as percent_client

from navegacao n
left join reserva r on r.id_cliente = n.id_cliente
''')

resposta = cursor.fetchall()

for resultado in resposta:
    percentual = resultado[0]  # Aqui pegamos o valor do percentual
    print(f"{int(percentual)}% de clientes realizam reservas após a navegação.")


62% de clientes realizam reservas após a navegação.


Existe alguma correlação entre o tempo gasto no site e a avaliação dada?

In [200]:
cursor.execute ('''

select

c.id_cliente
,n.tempo_gasto
,a.nota

from cliente c
left join navegacao n on n.id_cliente = c.id_cliente
left join avaliacao a on a.id_cliente = c.id_cliente
''')

resultados = cursor.fetchall()
colunas = [desc[0] for desc in cursor.description]
df = pd.DataFrame(resultados, columns=colunas)
correlacao = df['tempo_gasto'].corr(df['nota'])
print(f"A correlação entre tempo gasto no site e avaliação é: {correlacao:.2f}.")

A correlação entre tempo gasto no site e avaliação é: 0.02.


In [194]:
# Confirmar as alterações no bando de dados
conn.commit()

In [195]:
# Fechar a conexão após terminar todas as operações no banco de dados
conn.close()